In [1]:
import pandas as pd
import numpy as np
import regex
from pathlib import Path

HRSA_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\hrsa\HRSA - 2021 - Cleaned.csv')
srtr_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\otc-2009-raw.csv')


In [2]:
otc_2021 = HRSA_raw[['OTC Name', 'OTC Address', 'OTC City', 'Common State Name', 'Common State Abbreviation', 'OTC ZIP', 'OTC Longitude', 'OTC Latitude', 'OTC Code']]
otc_2021 = otc_2021.drop_duplicates().reset_index(drop=True)
otc_2021['OTC ZIP'] = otc_2021['OTC ZIP'].astype(object)

In [3]:
srtr_raw['OTC Address'] = srtr_raw['OTC Address'].astype(object)
srtr_raw['Common State Name'] = srtr_raw['Common State Name'].astype(object)
srtr_raw['OTC ZIP'] = srtr_raw['OTC ZIP'].astype(object)

In [4]:
srtr_pending = srtr_raw.loc[srtr_raw['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished = srtr_raw.loc[srtr_raw['OTC Code'].notnull()].reset_index(drop=True)

In [5]:
srtr_pending_manual = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\srtr_pending_manual.csv')

In [6]:
us_hospitals_full = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\us_hospital_locations.csv')

In [7]:
srtr_pending_2 = srtr_pending_manual.loc[srtr_pending_manual['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished_2 = srtr_pending_manual.loc[srtr_pending_manual['OTC Code'].notnull()].reset_index(drop=True)

In [8]:
us_hospitals = us_hospitals_full[['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'LONGITUDE', 'LATITUDE']]
us_hospitals['OTC Code'] = ''
us_hospitals = us_hospitals.rename(columns={'NAME' : 'OTC Name', 'ADDRESS' : 'OTC Address', 'CITY' : 'OTC City', 'STATE' : 'Common State Abbreviation', 'ZIP' : 'OTC ZIP', 'LONGITUDE' : 'OTC Longtitude', 'LATITUDE' : 'OTC Latitude'})
us_hospitals['key'] = us_hospitals['OTC Name'] + us_hospitals['OTC City'] + us_hospitals['Common State Abbreviation']

C:\Users\DAVIDC~1\AppData\Local\Temp/ipykernel_25576/1547666933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_hospitals['OTC Code'] = ''


In [9]:
srtr_pending_2['OTC Name'] = srtr_pending_2['OTC Name'].str.upper()
srtr_pending_2['OTC City'] = srtr_pending_2['OTC City'].str.upper()
srtr_pending_2['key'] = srtr_pending_2['OTC Name'] + srtr_pending_2['OTC City'] + srtr_pending_2['Common State Abbreviation']


In [10]:
srtr_keys = srtr_pending_2[['key']]
srtr_3 = srtr_keys.merge(us_hospitals, how='left', on='key')

In [11]:
srtr_pending_3 = srtr_3.loc[srtr_3['OTC Name'].isnull()].reset_index(drop=True)
srtr_finished_3 = srtr_3.loc[srtr_3['OTC Name'].notnull()].reset_index(drop=True)

In [12]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [13]:
match_key = []
match_ratio = []
OTC_Name = []
OTC_Address = []
OTC_City = []
Common_State_Abbreviation = []
OTC_ZIP = []
OTC_Longtitude = []
OTC_Latitude = []
confirmation = []

for i, r in srtr_pending_3.iterrows():
    name1 = r.key
    fuzzy = 0
    original = r['OTC Name']
    for x, y in us_hospitals.iterrows() :
        name2 = y.key
        wuzzy = fuzz.ratio(name1, name2)
        cOTC_Name = y['OTC Name']
        cOTC_Address = y['OTC Address']
        cOTC_City = y['OTC City']
        cCommon_State_Abbreviation = y['Common State Abbreviation']
        cOTC_ZIP = y['OTC ZIP']
        cOTC_Longtitude = y['OTC Longtitude']
        cOTC_Latitude = y['OTC Latitude']
        if wuzzy > fuzzy :
            fuzzy = wuzzy
            match = name2
            vOTC_Name = cOTC_Name
            vOTC_Address = cOTC_Address
            vOTC_City = cOTC_City
            vCommon_State_Abbreviation = cCommon_State_Abbreviation
            vOTC_ZIP = cOTC_ZIP
            vOTC_Longtitude = cOTC_Longtitude
            vOTC_Latitude = cOTC_Latitude
    match_key.append(match)
    match_ratio.append(fuzzy)
    OTC_Name.append(vOTC_Name)
    OTC_Address.append(vOTC_Address)
    OTC_City.append(vOTC_City)
    Common_State_Abbreviation.append(vCommon_State_Abbreviation)
    OTC_ZIP.append(vOTC_ZIP)
    OTC_Longtitude.append(vOTC_Longtitude)
    OTC_Latitude.append(vOTC_Latitude)
    confirmation.append(original)


In [14]:
srtr_finished_4 = pd.DataFrame(zip(confirmation, OTC_Name, OTC_Address, OTC_City, Common_State_Abbreviation, OTC_ZIP, OTC_Longtitude, OTC_Latitude, match_key, match_ratio), columns=['confirm', 'OTC Name', 'OTC Address', 'OTC City', 'Common State Abbreviation', 'OTC Zip', 'OTC Longtitude', 'OTC Latitude', 'Match', 'Ratio'])

In [15]:
srtr_finished_4['confirm'] = srtr_pending_3['OTC Name']

In [16]:
srtr_finished_4

,confirm,OTC Name,OTC Address,OTC City,Common State Abbreviation,OTC Zip,OTC Longtitude,OTC Latitude,Match,Ratio
0,NaN,BAPTIST MEDICAL CENTER - PRINCETON,"701 PRINCETON AVENUE, S.W.",BIRMINGHAM,AL,35211,-86.847188,33.498007,BAPTIST MEDICAL CENTER - PRINCETONBIRMINGHAMAL,98
1,NaN,JOHN L MCCLELLAN MEMORIAL VETERANS HOSPITAL,4300 WEST 7TH STREET,LITTLE ROCK,AR,72205,-92.320213,34.746036,JOHN L MCCLELLAN MEMORIAL VETERANS HOSPITALLIT...,79
2,NaN,PHOENIX INDIAN MEDICAL CENTER,4212 N 16TH STREET,PHOENIX,AZ,85016,-112.048858,33.497156,PHOENIX INDIAN MEDICAL CENTERPHOENIXAZ,90
3,NaN,VETERANS ADMINISTRATION MEDICAL CENTER,500 FOOTHILL DRIVE,SALT LAKE CITY,UT,84108,-111.840565,40.757441,VETERANS ADMINISTRATION MEDICAL CENTERSALT LAK...,79
4,NaN,ST. AGNES MEDICAL CENTER,1303 EAST HERNDON AVENUE,FRESNO,CA,93720,-119.765763,36.835087,ST. AGNES MEDICAL CENTERFRESNOCA,72
...,...,...,...,...,...,...,...,...,...,...
73,NaN,VETERANS ADMINISTRATION MEDICAL CENTER,500 FOOTHILL DRIVE,SALT LAKE CITY,UT,84108,-111.840565,40.757441,VETERANS ADMINISTRATION MEDICAL CENTERSALT LAK...,88
74,NaN,UNC LENOIR HEALTH CARE,100 AIRPORT ROAD,KINSTON,NC,28503,-77.583283,35.290341,UNC LENOIR HEALTH CAREKINSTONNC,64
75,NaN,ASCENSION ST FRANCIS HOSPITAL,3237 S 16TH ST,MILWAUKEE,WI,53215,-87.934657,42.985658,ASCENSION ST FRANCIS HOSPITALMILWAUKEEWI,62
76,NaN,VETERANS ADMINISTRATION MEDICAL CENTER,500 FOOTHILL DRIVE,SALT LAKE CITY,UT,84108,-111.840565,40.757441,VETERANS ADMINISTRATION MEDICAL CENTERSALT LAK...,77
